# Wikipedia feature engineering

The wikipedia part showcases our capabilities to extract features from Wikipedia using QIDs. Such features are gender, political assignation or age. Wikipedia data is quite messy and the heuristics used to extract these features are shown.

These features should prove useful to get interesting insights during Milestone 3.

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
from importlib import reload
import matplotlib.pyplot as plt
import urllib.request
import json
import sys
import re
import pickle

sys.path.append('../helpers/')
sys.path.append('../feature_engineering/')
import helpers

reload(helpers)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/lucastrg/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/lucastrg/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/lucastrg/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/lucastrg/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


<module 'helpers' from '/home/lucastrg/FLEP/MA1/ADA/ada-2021-project-adada-sur-mon-bidet/notebooks/../helpers/helpers.py'>

## Pre-Processing
We will remove all the quotes without a speaker, and we will extract the set of all the speakers and QIDs of the sampled rows.
We then fetch a json of each speaker's whole page as well as all its PIDs and RIDs (these 2 IDs are not yet in use)

In [4]:
df2 = pd.read_pickle("../data/WE_pref_df.pkl")


In [5]:
speakers=list(set(df2["speaker"]))

In [6]:
request_template="https://en.wikipedia.org/w/api.php?action=query&format=json&prop=revisions&titles={}&formatversion=2&rvprop=content&rvslots=*"


Doesn't work atm, not really useful so might drop

# Wikipedia data fetching
Fetches all we need to know about a speaker (using their name). Handles one redirection if needed 

In [7]:
invalid_speakers=[]
chunks_numbers=30
for i,speaker_chunk in enumerate(np.array_split(speakers,chunks_numbers)):

   speaker_cache={}
   speaker_mapping={}
   if i >=30: #Checkpoint
      for speaker in speaker_chunk:
         try :
            with urllib.request.urlopen(request_template.format(urllib.parse.quote(speaker))) as response:
               raw_data = json.load(response)["query"]["pages"][0]
               
               if raw_data.get("missing",False):
                  invalid_speakers.append(speaker)
               else:
                  content = raw_data["revisions"][0]["slots"]["main"]["content"]
                  if re.search("^'''{}''' may refer to".format(speaker),content): #Drop disambiguation pages
                     invalid_speakers.append(speaker)

                  else:
                     if re.search("(^#REDIRECT \[\[)([A-Za-z 'À-ÿZİı.-]*)", content): #Allows to fix most redirecting problems 
                        speaker_alt = re.search("(^#REDIRECT \[\[)([A-Za-z 'À-ÿZİı.-]*)", content).group(2)
                        #print("Redirect ", speaker ,"->",speaker_alt) #Je laisse le print parce qu'il est satisfaisant
                        if speaker_alt:
                           with urllib.request.urlopen(request_template.format(urllib.parse.quote(speaker_alt))) as response:
                              raw_data = json.load(response)["query"]["pages"][0]
                              if raw_data.get("missing",False):
                                 invalid_speakers.append(speaker)
                              else:
                                 content = raw_data["revisions"][0]["slots"]["main"]["content"]
                        else :
                           content = "ERROR"
                     speaker_mapping[speaker]=raw_data["title"]
                     speaker_cache[raw_data["title"]]=content
                  
         except urllib.request.HTTPError :
            invalid_speakers.append(speaker)
   with open('../speaker_cache/speaker_content_cache.pickle', 'ab') as handle:
      pickle.dump(speaker_cache, handle, protocol=pickle.HIGHEST_PROTOCOL)
   with open('../speaker_cache/speaker_mapping_cache.pickle', 'ab') as handle:
      pickle.dump(speaker_mapping, handle, protocol=pickle.HIGHEST_PROTOCOL)
      print("Checkpoint ", i)

Checkpoint  0
Checkpoint  1
Checkpoint  2
Checkpoint  3
Checkpoint  4
Checkpoint  5
Checkpoint  6
Checkpoint  7
Checkpoint  8
Checkpoint  9
Checkpoint  10
Checkpoint  11
Checkpoint  12
Checkpoint  13
Checkpoint  14
Checkpoint  15
Checkpoint  16
Checkpoint  17
Checkpoint  18
Checkpoint  19
Checkpoint  20
Checkpoint  21
Checkpoint  22
Checkpoint  23
Checkpoint  24
Checkpoint  25
Checkpoint  26
Checkpoint  27
Checkpoint  28
Checkpoint  29
